<h1 align="center"><font color="yellow">Quatro maneiras de Question-Answering in LangChain</font></h1>

<font color="yellow">Data Scientist.: Dr.Eddy Giusepe Chirinos Isidro</font>

Este script está baseado nos trabalhos da [Sophia Yang]()

# Contextualizando

Converse com seus longos documentos `PDF`: load_qa_chain, RetrievalQA, VectorstoreIndexCreator, ConversationalRetrievalChain.


Você está interessado em conversar com seus próprios documentos, seja um `arquivo de texto`, um `PDF` ou um `site`? O `LangChain` torna mais fácil para você responder a perguntas com seus documentos. Mas você sabia que existem pelo menos `4` maneiras de responder a perguntas no LangChain? Nesta postagem do blog, exploraremos quatro maneiras diferentes de responder a perguntas e as várias opções que você pode considerar para seus casos de uso.

Só lembrando que é `LangChain`, na opinião de `Sophia Yang`, é a maneira mais fácil de interagir com modelos de linguagem e criar aplicativos. É uma ferramenta de código aberto que envolve muitos `LLMs` e ferramentas. 

Ok, agora vamos começar a responder perguntas em documentos externos.


# Configurando a API da OpenAI e importando as Bibliotecas necessárias

Observe que a [API OpenAI](https://platform.openai.com/account) não é gratuita. Você precisará configurar as informações de cobrança para poder usar a `API OpenAI`. Como alternativa, você pode usar modelos do `HuggingFace Hub` ou de outros lugares.


In [ ]:
#%pip install langchain==0.0.137 openai chromadb tiktoken pypdf

In [3]:
# Isto é quando usas o arquivo .env:
from dotenv import load_dotenv
import os
print('Carregando a minha chave Key: ', load_dotenv())
Eddy_API_KEY_OpenAI = os.environ['OPENAI_API_KEY'] 

Carregando a minha chave Key:  True


In [4]:
from langchain.chains import RetrievalQA # Tem que atualizar --> pip install langchai==0.0.137
from langchain.llms import OpenAI
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma


# Carregando Documentos

O `LangChain` suporta muitos [carregadores de documentos](https://python.langchain.com/en/latest/modules/indexes/document_loaders.html), como `Notion`, `YouTube` e `Figma`. Neste exemplo, gostaria de conversar com meu arquivo `PDF`. Assim, usei o `PyPDFLoader` para carregar meu arquivo.

Eu recomendo usar um PDF pequeno. Eu usei, só para demonstração mesmo, um boleto de pagamento da minha conta de energia e um boleto de pagamento de mensalidade da minha faculdade que contém uma página cada boleto (você pode usar qualquer outro documento). Meu boleto contém certas entidades e em base a elas farei certas perguntas! 


In [5]:
# Verificamos nosso Modelo Instanciado:

llm = OpenAI()
print(llm("Conte-me uma piada."))




O que o oceano disse para o outro oceano? Nada, ele só estava ondulando.


In [6]:
# Carregando meu documento (Boleto de energia):
from langchain.document_loaders import PyPDFLoader

# loader = PyPDFLoader("./Karina_boleto_FACULDADE.pdf")  # PDF apenas com uma página
loader = PyPDFLoader("./Eddy_Running_the_Vale_API.pdf")
documents = loader.load()

In [7]:
documents

[Document(page_content='UFES - L abVISIO - V aleS.A.\nInstanciando a API de Análise de\nComunicação de Voz (ACV)\nData Scientist.: Dr.Eddy Giusepe Chirinos Isidro\n9 de março de 2023\n1', metadata={'source': './Eddy_Running_the_Vale_API.pdf', 'page': 0}),
 Document(page_content='Resumo\nNeste conciso tutorial aprenderemos a instanciar a nossa API de Análise\nde Comunicação de Voz (ACV) da Vale. Mostrarei os passos necessários\ne suficientes para poder executar a nossa ferramenta, comandos úteis de\nDockerização, a como expor um serviço local na Internet com ngrok e o\nuso de outras principais funcionalidades.\n2', metadata={'source': './Eddy_Running_the_Vale_API.pdf', 'page': 1}),
 Document(page_content='Conte ´udo\n1 Introdução 4\n2 Objetivo 5\n3 Principais passos para instanciar a API da Vale 6\n3.1 Ngrok . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 6\n3.1.1 Instalando o Ngrok . . . . . . . . . . . . . . . . . . . . . . . . . . 6\n3.2 docker compose . .

# Método 1: <font color="red">load_qa_chain</font>

`load_qa_chain` fornece a interface mais genérica para responder a perguntas. Ele carrega uma cadeia que você pode fazer `QA` para seus documentos de entrada e usa `TODO o texto` nos documentos.

`chain_type="stuff"` não funcionará porque o número de tokens excede o limite (isso para o caso do PDF da Sophia). Podemos tentar outros tipos de cadeia como `"map_reduce"`.

In [50]:
from langchain.chains.question_answering import load_qa_chain


chain = load_qa_chain(llm=OpenAI(),
                      chain_type="stuff"
                     )


query = "A pesquisa foi feita em que universidade?"

chain.run(input_documents=documents,
          question=query
         )


' A pesquisa foi feita na UFES.'

<font color="orange">Ele também permite que você faça QA em um conjunto de documentos:</font>

In [ ]:
### Para vários Documentos:
# loaders = [....]
# documents = []
# for loader in loaders:
#     documents.extend(loader.load())

<font color="red">🤔 Mas e se meu documento for muito longo e ultrapassar o limite de token?</font>

Existem duas maneiras de corrigi-lo:

## <font color="pink">Solução 1: Tipo de Cadeia (chain)</font>

O padrão `chain_type="stuff"` usa `TODO` o texto dos documentos no prompt. Se você tiver um arquivo muito grande (`tal como fez Sophia Yang, com um PDF de `50` páginas`) não funcionará porque excede o `limite de token` e causa erros de limitação de taxa. É por isso que para o exemplo da Sophia Yang, ela teve que usar outro tipo de cadeia, por exemplo `"map_reduce"`. 

A seguir vamos estudar outros tipos de cadeias:

🤩 `map_reduce:` Separa os textos em lotes (<font color="red">por exemplo</font>, você pode definir o tamanho do lote em `llm=OpenAI(batch_size=5)`), alimenta cada lote com a pergunta para o `LLM` separadamente e apresenta a resposta final com base nas respostas de cada lote.

🤩 `refine:` Separa os textos em lotes, alimenta o primeiro lote para o LLM e alimenta a resposta e o segundo lote para o LLM. Ele refina a resposta passando por todos os lotes.

🤩 `map_rerank:` ele separa os textos em lotes, alimenta cada lote para o LLM, retorna uma pontuação de como responde completamente à pergunta e apresenta a resposta final com base nas respostas com pontuação mais alta de cada lote.

## <font color="pink">Solução 2: Recuperação QA (`RetrievalQA`)</font>

Um problema com o uso de `TODO` o texto é que pode ser `muito caro` porque você está alimentando todos os textos para a `API OpenAI` e a `API` é cobrada pelo número de tokens. Uma solução melhor é recuperar os blocos (`chunks`) de texto relevantes primeiro e usar apenas os blocos de texto relevantes no modelo de linguagem. A seguir, examinarei os detalhes do `RetrievalQA`.

# Método 2: <font color="red">RetrievalQA</font>

<font color="orange">A cadeia `RetrievalQA` realmente usa `load_qa_chain` sob o capô. Recuperamos o pedaço de texto mais relevante e o alimentamos no modelo de linguagem.</font>

Opções (posteriormente descreveremos cada um delas):

* [Embeddings](https://python.langchain.com/en/latest/reference/modules/embeddings.html)

* [TextSplitter](https://python.langchain.com/en/latest/modules/indexes/text_splitters.html)

* [VectorStore](https://python.langchain.com/en/latest/modules/indexes/vectorstores.html)

* [Retrievers](https://python.langchain.com/en/latest/modules/indexes/retrievers.html)

  * [search_type](https://python.langchain.com/en/latest/modules/indexes/vectorstores/examples/chroma.html#mmr): "similarity" or "mmr"


* [Chain Type](https://python.langchain.com/en/latest/modules/chains/index_examples/question_answering.html): "stuff", "map reduce", "refine", "map_rerank"   




<font color="orange">A seguir vamos ver o como funciona, RetrievalQA:</font>


In [6]:
# Dividir os documentos em chunks:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)


# Selecione qual EMBEDDINGS quer usar:
embeddings = OpenAIEmbeddings()

# Crie o vectorestore para usar como índice (index):
db = Chroma.from_documents(texts, embeddings)


# Exponha este índice em uma interface de recuperação:
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k":1})


# Crie uma cadeia para responder perguntas:
qa = RetrievalQA.from_chain_type(
    llm=OpenAI(),
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True)

query = "Para que usam ngrok?"
result = qa({"query": query})


Using embedded DuckDB without persistence: data will be transient


<font color="orange">No resultado, podemos ver a resposta e dois documentos de origem porque definimos `k como 2`, o que significa que estamos interessados ​​apenas em obter dois chunks de texto relevantes.</font>

In [7]:
result

{'query': 'Para que usam ngrok?',
 'result': ' Ngrok é usado para criar um túnel seguro, atrás de Network Address Translation (NAT) e Firewalls, que expõem serviços locais (do computador) para a Internet, de forma fácil e segura.',
 'source_documents': [Document(page_content='3 Principais passos para instanciar a API daVale\nA seguir farei uma explanação dos principais passos para executar a nossa aplicação.\n3.1 N grok\nBasicamente, Ngrok é uma ferramenta CLI (Comand Line Interface) que te permite\ncriar um túnel seguro, atrás de Network Address Translation (NAT) e Firewalls, que\nexpõem serviços locais (do nosso computador) para a Internet, tudo isso de forma\nfácil e segura. A seguinte imagem, Figura 3.1, descreve o uso do ngrok [1]:\nFigura 3.1: Ngrok expondo um serviço local para a Internet.\n3.1.1 I nstalando o Ngrok\nO Ngrok é multiplataforma, ou seja, pode ser instalado no Linux ,Mac OS ou no\nWindows . Então, para usar o Ngrok o primeiro a fazer é criar uma conta no ngrok.com\

In [8]:
result['result']

' Ngrok é usado para criar um túnel seguro, atrás de Network Address Translation (NAT) e Firewalls, que expõem serviços locais (do computador) para a Internet, de forma fácil e segura.'

## <font color="red">Usando similaridade com score no CHROMA</font>

In [22]:
# Dividir os documentos em chunks:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)


# Selecione qual EMBEDDINGS quer usar:
embeddings = OpenAIEmbeddings()

# Crie o vectorestore para usar como índece (index):
db = Chroma.from_documents(texts, embeddings)



docs_score = db.similarity_search_with_score("O que é system prune?")

docs_score

Using embedded DuckDB without persistence: data will be transient


[(Document(page_content='•Para executar o seguinte comando devemos estar no diretório onde se encontra\no arquivo docker-compose.yml . Ver a Figura 3.3, acima.\n$ docker compose build\nEste comando e basicamente usado para construir ou reconstruir as imagens\nDocker definidas no arquivo docker-compose.yml.\n•O seguinte comando é usado para construir, criar ou recriar e executar con-\ntêineres para os serviços definidos no arquivo docker-compose.yml , também\nconfigura as redes e os volumes especificados em dito arquivo.\n$ docker compose up\nEste comando além de iniciar os contêineres em um modo interativo e exibir\na saída do console para cada contêiner, permite que você monitore o status do\ncontêiner, visualize eventuais erros e saiba quando o serviço estiver pronto para\nuso.\nÉ importante aprender, também, como parar nossa aplicação. Então, para interromper\nos contêineres de maneira segura, você pode executar (em outro terminal) o seguinte\ncomando:\n$ docker compose down\nE para

In [23]:
max_value = max(docs_score, key=lambda x: x[1])
max_value

(Document(page_content='Resumo\nNeste conciso tutorial aprenderemos a instanciar a nossa API de Análise\nde Comunicação de Voz (ACV) da Vale. Mostrarei os passos necessários\ne suficientes para poder executar a nossa ferramenta, comandos úteis de\nDockerização, a como expor um serviço local na Internet com ngrok e o\nuso de outras principais funcionalidades.\n2', metadata={'source': './Eddy_Running_the_Vale_API.pdf', 'page': 1}),
 0.4965440034866333)

## Opções

Existem várias opções para você escolher neste processo:


🤗 [embeddings](https://python.langchain.com/en/latest/reference/modules/embeddings.html): No exemplo, usamos `OpenAI Embeddings`. Mas existem muitas outras opções de Embeddings, como `Cohere Embeddings` e `HuggingFaceEmbeddings` de modelos específicos.

🤗 [TextSplitter](https://python.langchain.com/en/latest/modules/indexes/text_splitters.html): Usamos o `Character Text Splitter` no exemplo em que o texto é dividido por um único caractere. Você também pode usar diferentes divisores de texto e diferentes `tokens` mencionados neste documento.

🤗 [VectorStore](https://python.langchain.com/en/latest/modules/indexes/vectorstores.html): Usamos o `Chroma` como nosso `banco de dados de vetores`, onde armazenamos nossos vetores de texto incorporados. Outras opções populares são `FAISS`, `Mulvus` e `Pinecone`.

🤗 [Retrievers](https://python.langchain.com/en/latest/modules/indexes/retrievers.html): usamos um `VectoreStoreRetriver`, que é apoiado por um `VectorStore`. Para recuperar o texto, existem dois tipos de pesquisa que você pode escolher: [search_type](https://python.langchain.com/en/latest/modules/indexes/vectorstores/examples/chroma.html#mmr):"similarity" ou “mmr”. `search_type="similarity"` usa pesquisa de similaridade no objeto retriever, onde seleciona vetores de blocos (chunk) de texto que são mais similares ao vetor de pergunta. `search_type="mmr"` usa a pesquisa de `relevância marginal máxima` (mmr - maximum marginal relevance) onde otimiza a similaridade para query `AND` a diversidade entre os documentos selecionados.

🤗 [Chain Type](https://python.langchain.com/en/latest/modules/chains/index_examples/question_answering.html): igual ao `método 1`. Você também pode definir o tipo de cadeia como uma das quatro opções: `“stuff”`, `“map reduce”`, `“refine”`, `“map_rerank”`.



# Método 3: <font color="red">VectorstoreIndexCreator</font>

`VectorstoreIndexCreator` é um `wrapper` em torno da funcionalidade acima. É exatamente o mesmo sob o capô, mas apenas expõe uma interface de nível superior para permitir que você comece em três linhas de código:

In [54]:
index = VectorstoreIndexCreator().from_loaders([loader])

query = "Qual foi o objetivo do trabalho de pesquisa?"

index.query(llm=OpenAI(),
            question=query,
            chain_type="map_reduce"
           )


Using embedded DuckDB without persistence: data will be transient


' O objetivo do trabalho de pesquisa foi conhecer e aplicar os comandos necessários para instanciar a aplicação da Vale, que realiza a segmentação de um áudio (Diarization), a transcrição de um áudio (speech-to-text) e a identificação de quem fala num determinado trecho de áudio (Speaker Identification).'

<font color="orange">Claro, você também pode especificar diferentes opções neste `wrapper`:

</font>

In [58]:
index = VectorstoreIndexCreator(
    # Dividir os documentos em chunks
    text_splitter=CharacterTextSplitter(chunk_size=1000, chunk_overlap=0),
    # Selecione os Embeddings que quer usar:
    embedding=OpenAIEmbeddings(),
    # Use Chroma como o vectorestore para indexar e pesquisar Embeddings:
    vectorstore_cls=Chroma
).from_loaders([loader])


query = "Quem foi o Cientista de Dados?"

index.query(llm=OpenAI(),
            question=query,
            chain_type= "map_rerank" #"stuff" # Usando `map_reduce` não trouxe resposta a minha pergunta.` 
           )




Using embedded DuckDB without persistence: data will be transient


' Dr.Eddy Giusepe Chirinos Isidro'

# Método 4: <font color="red">ConversationalRetrievalChain</font>

`ConversationalRetrievalChain` é muito semelhante ao `método 2 RetrievalQA`. Ele adicionou um parâmetro adicional `chat_history` para passar no histórico do bate-papo, que pode ser usado para perguntas de acompanhamento.

```
ConversationalRetrievalChain = memória de conversação + RetrievalQAChain
```

<font color="yellow">Se você quiser que seu modelo de linguagem tenha uma memória da conversa anterior, use este método.</font> 


No meu exemplo a seguir, perguntei para que foi importante aprender certos comandos no trabalho de pesquisa?. Como ele tem todo o histórico do chat, me trouxe uma boa resposta.

In [3]:
from langchain.chains import ConversationalRetrievalChain


# Carregamos nosso documento:
loader = PyPDFLoader("./Eddy_Running_the_Vale_API.pdf")
documents = loader.load()

# Dividir os Documentosem Chunks:
text_splitter = CharacterTextSplitter(chunk_size=1000,
                                      chunk_overlap=0
                                     )

texts = text_splitter.split_documents(documents)


# Seleciono os Embeddings que queremos usar:
embeddings = OpenAIEmbeddings()

# Criamos o vectorestore para usar como índice:
db = Chroma.from_documents(texts, embeddings)

# Exponha este índice em uma interface de recuperação:
retriever = db.as_retriever(search_type="similarity",
                            search_kwargs={"k":2}
                           )

# Criamos uma cadeia (chain) para responder a perguntas:  
qa = ConversationalRetrievalChain.from_llm(OpenAI(),
                                           retriever
                                          )


Using embedded DuckDB without persistence: data will be transient


In [4]:
chat_history = []

query = "Qual foi o objetivo do trabalho de pesquisa?"
result = qa({"question": query, "chat_history": chat_history})


In [5]:
result

{'question': 'Qual foi o objetivo do trabalho de pesquisa?',
 'chat_history': [],
 'answer': ' Conhecer e aplicar os comandos necessários para instanciar a aplicação da Vale.'}

In [6]:
result['answer']

' Conhecer e aplicar os comandos necessários para instanciar a aplicação da Vale.'

In [7]:
chat_history = [(query, result["answer"])]

query = "Com os comandos aprendidos se instanciou o que?"
result = qa({"question": query, "chat_history": chat_history})

In [8]:
chat_history

[('Qual foi o objetivo do trabalho de pesquisa?',
  ' Conhecer e aplicar os comandos necessários para instanciar a aplicação da Vale.')]

In [9]:
result["answer"]

' A API de Análise de Comunicação de Voz (ACV) da Vale.'

# Conclusão

<font color="orange">Agora você conhece quatro maneiras de responder a perguntas com `LLMs` no `LangChain`. Em resumo, `load_qa_chain` usa todos os textos e aceita vários documentos; O `RetrievalQA` usa **load_qa_chain** sob o capô, mas recupera primeiro os blocos de texto relevantes; `VectorstoreIndexCreator` é o mesmo que **RetrievalQA** com uma interface de nível superior; `ConversationalRetrievalChain` é útil quando você deseja passar seu `histórico de bate-papo` para o modelo.</font>


Thanks God!